In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb

!pip install cvxpy
!pip install box2d-py
!pip uninstall pyglet -y
!pip uninstall gym -y
!pip install tensorflow==1.15
!pip install pyglet==1.3.2
!pip install gym==0.9.4
!pip install pyvirtualdisplay

In [ ]:
!git clone -b paper-training https://github.com/naufalhisyam/rocket-lander.git
%cd /content/rocket-lander
!ls

In [ ]:
# Virtual display
from pyvirtualdisplay import Display
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

virtual_display = Display(visible=0, size=(1000, 800))
virtual_display.start()

def show_video():
  mp4list = glob.glob('rocket_videos/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

In [ ]:

%tensorflow_version 1.x
import numpy as np
from numpy.core.numeric import False_
import pandas as pd
import tensorflow as tf

from control_and_ai.DDPG.ddpg import DDPG
from control_and_ai.DDPG.utils import Utils
from control_and_ai.DDPG.exploration import OUPolicy

from constants import *
from constants import DEGTORAD
from environments.rocketlander_test import RocketLander, get_state_sample

action_bounds = [1, 1, 15*DEGTORAD]

eps = []
eps.append(OUPolicy(0, 0.2, 0.4))
eps.append(OUPolicy(0, 0.2, 0.4))
eps.append(OUPolicy(0, 0.2, 0.4))

simulation_settings = {'Side Engines': True,
                       'Clouds': True,
                       'Vectorized Nozzle': True,
                       'Graph': False,
                       'Render': False,
                       'Starting Y-Pos Constant': 1,
                       'Initial Force': 'random',
                       'Rows': 1,
                       'Columns': 2,
                       'Episodes': 500}

env = RocketLander(simulation_settings)
env = wrappers.Monitor(env, './rocket_videos', force=True)

#Set both line below to False if you want to contniue training from a saved checkpoint
RETRAIN = False#Restore weights if False
TEST = True #Test the model

NUM_EPISODES = 1
SAVE_TO_EXCEL = True #Export states & actions logs as .xlsx

NAME = "test" #Model name

SIMULATE_WIND = True
x_force = 2000 # x-axis wind force in Newton
y_force = 2000 # y-axis wind force in Newton

model_dir = '/content/gdrive/MyDrive/colab_model/rocket/DDPG/' + NAME

agent = DDPG(
    action_bounds,
    eps,
    env.observation_space.shape[0], #for first model
    actor_learning_rate=0.0001,
    critic_learning_rate=0.001,
    retrain=RETRAIN,
    log_dir=logs,
    model_dir=model_dir,
    batch_size=100,
    gamma=0.99)

In [ ]:
def test(env, agent, x_force, y_force):
    obs_size = env.observation_space.shape[0]

    util = Utils()
    state_samples = get_state_sample(samples=5000, normal_state=True)
    util.create_normalizer(state_sample=state_samples)
    if SAVE_TO_EXCEL:
        xpos, ypos, xvel, yvel, lander_angle, angular_vel, rem_fuel, lander_mass = ([] for _ in range(8))
        fE, fS, pSi = ([] for _ in range(3))

    for episode in range(1, NUM_EPISODES + 1):
        old_state = None
        done = False
        total_reward = 0

        state = env.reset()
        state = util.normalize(state)
        max_steps = 500

        left_or_right_barge_movement = np.random.randint(0, 2)

        for t in range(max_steps): # env.spec.max_episode_steps
            old_state = state
            # infer an action
            action = agent.get_action(np.reshape(state, (1, obs_size)), not TEST)
            
            if SAVE_TO_EXCEL:
                current_state = env.get_state_with_barge_and_landing_coordinates(untransformed_state=True)
                
                xpos.append(current_state[0]-current_state[12]) #xpos_rocket - xpos_landingPad
                ypos.append(current_state[1]-current_state[13]) #ypos_rocket - ypos_landingPad
                xvel.append(current_state[2]) #xdot
                yvel.append(current_state[3]) #ydot
                lander_angle.append(current_state[4]) #theta
                angular_vel.append(current_state[5]) #theta_dot
                rem_fuel.append(current_state[6]) #initial fuel = 0.2 * initial_mass
                lander_mass.append(current_state[7]) #initial_mass = 25.222
                
                fE.append(action[0][0])
                fS.append(action[0][1])
                pSi.append(action[0][2])

            # take it
            state, reward, done, _ = env.step(action[0])
            state = util.normalize(state)
            total_reward += reward

            if SIMULATE_WIND:
                if state[LEFT_GROUND_CONTACT] == 0 and state[RIGHT_GROUND_CONTACT] == 0:
                    env.apply_random_x_disturbance(epsilon=0.005, left_or_right=left_or_right_barge_movement, x_force=x_force)
                    env.apply_random_y_disturbance(epsilon=0.005, y_force=y_force)

            if not TEST:
                # update q vals
                agent.update(old_state, action[0], np.array(reward), state, done)

            if done:
                break

        agent.log_data(total_reward, episode)

        if episode % 50 == 0 and not TEST:
            print('Saved model at episode', episode)
            agent.save_model(episode)
        print("Episode:\t{0}\tReward:\t{1}".format(episode, total_reward))
    
    if SAVE_TO_EXCEL:
        state_data=pd.DataFrame(list(zip(xpos,ypos,xvel,yvel,lander_angle,angular_vel,rem_fuel,lander_mass)),\
            columns=['x_pos','y_pos','x_vel','y_vel','lateral_angle','angular_velocity','remaining_fuel','lander_mass'])
        action_data=pd.DataFrame(list(zip(fE,fS,pSi)),columns=['Fe','Fs','Psi'])
        with pd.ExcelWriter(f"/content/rocket-lander/excel_logs/states-acts/DDPG_{NAME}_{total_reward}.xlsx") as writer:
            state_data.to_excel(writer, sheet_name="state")
            action_data.to_excel(writer, sheet_name="action")
        !cp -a "/content/rocket-lander/excel_logs/states-acts/." "{model_dir}"
        !cp -a "/content/rocket-lander/excel_logs/rocket_videos/." "{model_dir}"
        print("Logs saved")
    env.close()
    show_video()

In [ ]:
test(env, agent, x_force, y_force)